In [ ]:
!nvidia-smi

In [ ]:
%%capture
!pip install datasets
!pip install transformers
!pip install rouge-score
!pip install nltk

In [ ]:
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration
from datasets import load_dataset, load_metric
import torch
import random
import nltk
from nltk import sent_tokenize

In [ ]:
nltk.download('punkt')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
rouge = load_metric("rouge", seed=2022)

In [ ]:
model_checkpoint = 'microsoft/prophetnet-large-uncased-cnndm'

In [ ]:
tokenizer = ProphetNetTokenizer.from_pretrained(model_checkpoint)

model = ProphetNetForConditionalGeneration.from_pretrained(model_checkpoint)
model.to(device)

In [ ]:
model.num_parameters()

In [ ]:
data_test = load_dataset('cnn_dailymail', '3.0.0', split='test')

In [ ]:
batch_size = 16

In [ ]:
# map data correclty
def generate_summary(batch):
    inputs = tokenizer(batch['article'],
                       padding=True,
                       truncation=True,
                       max_length=512,
                       add_special_tokens=False,
                       return_tensors='pt')
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # The following decoding parameters are set according to the ProphetNet paper
    outputs = model.generate(input_ids,
                             attention_mask=attention_mask,
                             min_length=45,  # apo to paper? nai
                             max_length=110, # apo to paper? nai
                             num_beams=5, # json.config & paper
                             no_repeat_ngram_size=3, # json.config & paper
                             length_penalty=1.2, # json.config & paper
                             early_stopping=True # json.config
                             )
    

    # all special tokens will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch['pred'] = output_str

    return batch

In [ ]:
results = data_test.map(generate_summary,
                        batched=True,
                        batch_size=batch_size,
                        remove_columns=['article'])

labels = results['highlights']
predictions = results['pred']

In [ ]:
# ROUGE expects a newline after each sentence
clean_preds = ["\n".join(sent_tokenize(pred.replace('[X_SEP]', ' '))) for pred in predictions]
clean_labels = [label.replace(" .", ".") for label in labels]
# clean_labels = ["\n".join(sent_tokenize(label.replace(" .", "."))) for label in labels]

for i in range(len(predictions)):
    print(f"Item {i}:")
    print(clean_preds[i])
    print("\n")
    print(clean_labels[i])
    print("\n")

In [ ]:
rouge_output = rouge.compute(predictions=clean_preds, references=clean_labels, use_stemmer=True)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(rouge_output[rn].mid.fmeasure * 100, 2)) for rn in rouge_names)

print(rouge_dict)